In [1]:
import remi.gui as gui
from remi import start, App
from threading import Timer, Thread

In [2]:
class Proxy():
    value = True
    @staticmethod
    def set_url(url):
        if url.startswith("data:"):
            return url
        else:
            return "/proxy/8085"+url
    @staticmethod
    def get_url():
        return "/proxy/8085"  

In [3]:
class MyApp(App):
    def __init__(self, *args):
        super(MyApp, self).__init__(*args)

    def main(self):
        #creating a container VBox type, vertical
        wid = gui.VBox(width=300, height=200)

        #creating a text label
        self.lbl = gui.Label('Hello', width='80%', height='50%')

        #a button for simple interaction
        bt = gui.Button('Press me!', width=200, height=30)

        #setting up the listener for the click event
        bt.onclick.connect(self.on_button_pressed)
        
        #adding the widgets to the main container
        wid.append(self.lbl)
        wid.append(bt)

        # returning the root widget
        return wid

    # listener function
    def on_button_pressed(self, emitter):
        # print("button pressed")
        self.lbl.set_text('Hello World!')

In [4]:
myRemi = Thread(target=start, 
                         args=(MyApp,),
                         kwargs={'address':'127.0.0.1', 
                                 'port':8085, 
                                 'multiple_instance':True,
                                 'enable_file_cache':True, 
                                 'update_interval':0.5, 
                                 'start_browser':False,
                                 'proxy':Proxy})
myRemi.start()  

remi.server      INFO     Started httpserver http://127.0.0.1:8085/
remi.request     INFO     built UI (path=/)
127.0.0.1 - - [22/Nov/2021 18:21:36] "GET / HTTP/1.1" 200 -
remi.server.ws   INFO     connection established: ('127.0.0.1', 65377)
remi.server.ws   INFO     handshake complete
remi.server.ws   ERROR    Error managing incoming websocket message
Traceback (most recent call last):
  File "c:\Users\gilbe\Documents\GitHub\remi\venv\lib\site-packages\remi\server.py", line 166, in read_next_message
    length = self.bytetonum(length[1]) & 127
IndexError: index out of range


http://127.0.0.1:8888/proxy/8085/